# Week1

- train/Dev/Test Set
- 利用Bias和Variance诊断模型
- 正则化
    - L1/L2
    - Dropout
    - others
- 归一化
- 梯度爆炸和梯度消失
    - Xavier Initialization

## 训练集, 验证集, 测试集关系

- 训练集: 用训练集对算法或模型进行训练过程
- 验证集: 利用验证集或者又称为简单交叉验证集进行交叉验证，选择出最好的模型
- 测试集: 最后利用测试集对模型进行测试，获取模型运行的无偏估计。

> **为什么会有验证集?**<br/>
测试集不能被提前窥探, 训练集在训练过程中已经被一定程度拟合, 不能作为调参依据, 人工调参时需要使用没有被训练过的数据比较不同参数的模型的好坏, 这部分数据由于被人工调参过程中进行了一定程度拟合, 不能作为模型泛化性能的评价标准

k折交叉验证:

1. 随机将训练数据等分成k份，$S_1, S_2,\dots, S_k$
2. 对于每一个模型$M_i$，算法执行k次，每次选择一个$S_j$作为验证集，而其它作为训练集来训练模型$M_i$，把训练得到的模型在$S_j$上进行测试，这样一来，每次都会得到一个误差E，最后对k次得到的误差求平均，就可以得到模型$M_i$的泛化误差。
![交叉验证](src/pictures/c2_w1_1.png)

K值的选择

K值的选取是一个偏差与方差的权衡：
- K=1时，所有数据用于训练，容易过拟合
- K=N时，相当于留一法LOOCV, 通常建议K=10, 或$K\approx log(n)$


## 利用Bias和Variance诊断模型

模型在欠拟合的情况下，出现高偏差；在过拟合的情况下，出现高方差. 在bias-variance tradeoff 的角度, 利用训练集对模型进行训练就是为了使得模型在train集上使**bias**最小化，避免出现underfitting的情况; 模型设置的太复杂则没有排除一些train集数据中的噪声，使得模型出现overfitting的情况，在dev 集上出现高**variance**的现象.

### 处理方法

1. 是否存在High bias ? 
    - 增加网络结构，如增加隐藏层数目
    - 训练更长时间
    - 寻找合适的网络架构，使用更大的NN结构
2. 是否存在High variance？ 
    - 获取更多的数据
    - 正则化(regularization)
    - 寻找合适的网络结构

## 正则化

目的是解决高方差问题, 通过在目标函数中加入度量模型复杂度的项惩罚模型的复杂度, 获得更好的泛化性.

### L1/L2正则化

对模型的参数取L1/L2范数加入到最小化的目标函数中, 迫使模型更加平滑.

- L1: $\frac{\lambda}{2m}||w||_1 = \frac{\lambda}{2m} \sum_{j=1}^{n_x}|w_j|$
- L2: $\frac{\lambda}{2m}||w||_2^2 = \frac{\lambda}{2m} \sum_{j=1}^{n_x} w_j^2$

其中$\lambda$为正则化因子, 为超参.

加入L2正则后, 参数梯度的更新式中$W=W-\alpha(gradient)$变为了$W = (1-\frac{\alpha \lambda}{m})W - \alpha(gradient)$, 因此L2范数正则化称为权重衰减.

> 直观解释:<br/>
W项减小导致Z减小, 使用tanh作为激活函数时,Z在较小范围变化, tanh(z)函数近似线性, 每层函数近似线性函数, 则整个网络更加简单, 从而降低过拟合的情况.

### Dropout

为每个神经网络设置随机消除的概率, 得到一个节点更少规模更小的网络进行训练. 实现方法, 反向随机失活, **注意事项**是恢复失活后下一层的期望.

>为什么Dropout有效?<br/>
输入特征可能被随机清除, 所以神经元不能够特别依赖任何一个输入特征, 不会给任何一个输入设置太大的权重. 和L2范数相同具有收缩权重的效果.

缺点: 使得cost function不能被明确定义. 建议首先关闭Dropout能够保证Cost函数单调递减, 之后再开启提高泛化能力.

### 其他方法

- 数据增广: 对图像进行裁剪, 变换得到新的数据集和验证集
- 早停: 在交叉验证集的误差上升的点停止迭代, 防止过拟合. 缺点是无法同时解决Bias和Var之间的最优.

## 归一化

目的是加速收敛, 能够使用更大的学习率进行训练.

数据归一化步骤:
- 计算所有样本每一维均值$\mu$
- 减去均值得到对称分布, $x = x -\mu$
- 归一化方差$\sigma^2 = \frac{1}{m}\sum_{i=1}^m x^{(i)2}, x=x/\sigma^2$

![归一化](src/pictures/c2_w1_2.png)

## 梯度消失或梯度爆炸

对于一定层数以上的神经网络容易出现的问题, 梯度计算时, 根据情况不同可能会出现梯度函数以指数递增或递减的情况,导致训练步长变得非常小, 难以收敛. 梯度爆炸问题可以通过clipping方式解决, 即梯度大于某阈值时, 设置为该值, 防止梯度的增长继续传递. 梯度消失的问题比较不容易解决, 需要根据模型讨论.

### 初始化角度缓解

对于一个神经元, 当输入的数量较大时, 希望每个$w_i$值都小一些, 这样z也比较小, 为了得到较小的$w_i$, 可以对其按照$Var(W_i)=\frac{1}{n}$进行缩放, 称为**Xavier initialization**.

> 为什么可行?<br/>
如果激活函数的输入x近似设置成均值0, 标准差1的情况, 输出z也会调整到相似的范围. 在一定程度上缓解梯度消失和爆炸速速.